In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4549/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4549/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_3_9_6,0.972722,0.887735,0.928797,0.974172,0.178611,0.750716,0.444747,0.178486,0.754941,0.422624,0.952158,0.426348,101.445089,235.946764,"Hidden Size=[12], regularizer=0.02, learning_r..."
1,model_3_41_4,0.981539,0.885759,0.932348,0.972092,0.120877,0.763929,0.422571,0.266175,0.449292,0.347674,0.967623,0.350737,102.225962,236.727636,"Hidden Size=[12], regularizer=0.02, learning_r..."
2,model_3_10_5,0.977595,0.885533,0.919370,0.974090,0.146700,0.765440,0.503632,0.158272,0.650542,0.383014,0.960706,0.386389,101.838732,236.340406,"Hidden Size=[12], regularizer=0.02, learning_r..."
3,model_3_10_9,0.976705,0.885194,0.916182,0.965984,0.152533,0.767708,0.523545,0.207784,0.674177,0.390555,0.959143,0.393996,101.760745,236.262420,"Hidden Size=[12], regularizer=0.02, learning_r..."
4,model_3_10_1,0.976218,0.884977,0.923177,0.977311,0.155717,0.769157,0.479856,0.138594,0.716139,0.394609,0.958291,0.398086,101.719436,236.221111,"Hidden Size=[12], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782,model_5_13_0,0.935907,-0.135747,-0.222379,0.945080,0.419666,7.594746,7.635244,0.373709,0.774351,0.647816,1.235696,0.653524,291.736595,689.751753,"Hidden Size=[36], regularizer=0.02, learning_r..."
783,model_5_12_6,0.932867,-0.142020,-0.214864,0.951221,0.439573,7.636690,7.588301,0.360558,0.814891,0.663003,1.246876,0.668844,291.643904,689.659063,"Hidden Size=[36], regularizer=0.02, learning_r..."
784,model_5_12_2,0.927746,-0.154217,-0.213696,0.949629,0.473106,7.718255,7.581007,0.372325,0.889425,0.687827,1.265710,0.693887,291.496873,689.512031,"Hidden Size=[36], regularizer=0.02, learning_r..."
785,model_5_12_0,0.924606,-0.161618,-0.213151,0.946943,0.493666,7.767747,7.577602,0.392180,0.935119,0.702613,1.277257,0.708804,291.411794,689.426952,"Hidden Size=[36], regularizer=0.02, learning_r..."
